In [1]:
%cd ../..

/home/eli/AnacondaProjects/epych


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import epych
import glob
import pynwb
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import quantities as pq
import scipy

In [4]:
%matplotlib inline

In [5]:
logging.basicConfig(level=logging.INFO)

In [6]:
def epoch_intervals(intervals, epoch):
    mask = intervals[epoch][:].astype(bool)
    return np.stack((intervals['start_time'][mask], intervals['stop_time'][mask]), axis=-1)

In [7]:
def trial_intervals(intervals):
    trial_nums = intervals['trial_num'][:].astype(int)
    trials = np.unique(trial_nums)
    startstops = []
    for trial in trials:
        indices = np.nonzero(trial_nums == trial)[0]
        startstops.append((intervals['start_time'][indices[0]], intervals['stop_time'][indices[-1]]))
    return np.array(startstops)

In [8]:
def trial_stimulus_intervals(intervals):
    trial_nums = intervals['trial_num'][:].astype(int)
    trials = np.unique(trial_nums)
    stimuli = {k: [] for k in range(5)}
    for trial in trials:
        for index, interval in enumerate(np.nonzero(trial_nums == trial)[0]):
            stimuli[index].append((intervals['start_time'][interval], intervals['stop_time'][interval]))
    return {k: np.array(v) for k, v in stimuli.items()}

In [9]:
CONDITIONS = ["go_gloexp", "lo_gloexp", "go_seqctl", "lo_rndctl", "igo_seqctl"]
PRETRIAL_SECONDS = 0.5 * pq.second
POSTTRIAL_SECONDS = 0.5 * pq.second

In [10]:
def hippocampal_areas(probe_areas):
    for (c, a) in enumerate(probe_areas):
        if "DG-" in a or "CA" in a:
            yield (c, a)

def visual_areas(probe_areas):
    for (c, a) in enumerate(probe_areas):
        if "VIS" in a:
            yield (c, a)

def subcortical_areas(probe_areas):
    for (c, a) in enumerate(probe_areas):
        if "DG-" in a or "CA" in a or "MB" in a or "SCi" in a or "POST" in a:
            yield (c, a)

In [11]:
def probe_electrode_metadata(electrodes, probe, channels):
    indices = (electrodes['group_name'][:] == probe.encode()).nonzero()[0]
    return {
        'areas': np.array([area.decode() for area in electrodes['location'][indices][channels]]),
        'horizontal': electrodes['probe_horizontal_position'][indices][channels] * pq.mm,
        'vertical': electrodes['probe_vertical_position'][indices][channels] * pq.mm,
    }

In [12]:
NWB_SUBJECTS = glob.glob('/mnt/data/DRAFT/000253/sub-*/')

In [13]:
PILOT_FILES = []
REQUIRED_PROBES = 6

In [14]:
def probe_lfps(nwb_files, electrodes):
    signals = {}
    for p, nwb_file in enumerate(nwb_files):
        with pynwb.NWBHDF5IO(path=nwb_file, mode="r") as io:
            nwb = io.read()

            probe_lfp = "probe_%d_lfp_data" % p
            lfps = nwb.acquisition[probe_lfp]
            timestamps = lfps.timestamps[:] * pq.second
            dt = (timestamps[1:] - timestamps[:-1]).mean()
            if dt > 1.:
                raise ValueError('Invalid timestamps!')

            probe = list(nwb.electrode_groups.keys())[0]
            probe_electrodes = electrodes["group_name"][:] == probe
            channels = {
                "horizontal": electrodes["probe_horizontal_position"][probe_electrodes] * pq.mm,
                "id": electrodes["id"][probe_electrodes],
                "location": electrodes["location"][probe_electrodes],
                "vertical": electrodes["probe_vertical_position"][probe_electrodes] * pq.mm,
            }
            channels = {k: v[lfps.electrodes['local_index'][:]] for k, v in channels.items()}
            probe_channels = pd.DataFrame(data=channels, columns=["horizontal", "location", "vertical"], index=channels['id'])
            signals[probe] = epych.signals.lfp.RawLfp(probe_channels, lfps.data[:] * pq.volt, dt, timestamps, channels_dim=1, time_dim=0)
    return signals

In [15]:
def passiveglo_epochs(glo_intervals):
    epochs = {
        "start": np.array([]),
        "end": np.array([]),
        "type": [],
    }

    trials = trial_intervals(glo_intervals)
    trial_times = trials[:, 0]
    epochs["start"] = np.concatenate((epochs["start"], trials[:, 0].squeeze()), axis=0)
    epochs["end"] = np.concatenate((epochs["end"], trials[:, 1].squeeze()), axis=0)
    epochs["type"] = epochs["type"] + ["trial"] * trials.shape[0]

    for condition in CONDITIONS:
        cond_times = epoch_intervals(glo_intervals, condition)
        epochs["start"] = np.concatenate((epochs["start"], cond_times[:, 0].squeeze()), axis=0)
        epochs["end"] = np.concatenate((epochs["end"], cond_times[:, 1].squeeze()), axis=0)
        epochs["type"] = epochs["type"] + [condition] * cond_times.shape[0]

    stim_times = trial_stimulus_intervals(glo_intervals)
    for k, v in stim_times.items():
        epochs["start"] = np.concatenate((epochs["start"], v[:, 0].squeeze()), axis=0)
        epochs["end"] = np.concatenate((epochs["end"], v[:, 1].squeeze()), axis=0)
        epochs["type"] = epochs["type"] + ["stim%d" % k] * v.shape[0]

    return pd.DataFrame(data=epochs, columns=epochs.keys())

In [16]:
def subjectdir_recording(subject_dir):
    passive_glo = scipy.io.loadmat(subject_dir + 'passiveglo_task_data.mat')
    epochs = passiveglo_epochs(passive_glo)
    units = {"start": pq.second, "end": pq.second}
    ogen_nwb = glob.glob(subject_dir + '/sub-*_ogen.nwb')[0]
    with pynwb.NWBHDF5IO(path=ogen_nwb, mode="r") as io:
        nwb = io.read()
        electrodes = {}
        electrodes['group_name'] = nwb.electrodes.group_name[:]
        electrodes['id'] = nwb.electrodes.id[:]
        electrodes['location'] = nwb.electrodes.location[:]
        electrodes['probe_horizontal_position'] = nwb.electrodes.probe_horizontal_position[:] * pq.mm
        electrodes['probe_vertical_position'] = nwb.electrodes.probe_vertical_position[:] * pq.mm

    nwb_files = sorted(glob.glob(subject_dir + '/sub-*_ses-*_probe-*.nwb'))
    nwb_files = [nwbf for nwbf in nwb_files if "test" not in nwbf]
    if len(nwb_files) < REQUIRED_PROBES:
        PILOT_FILES.append(subject_dir)
        return None

    try:
        signals = probe_lfps(nwb_files, electrodes)
        return epych.recording.RawRecording(epochs, pd.DataFrame(columns=["trial"]).set_index("trial"), units, **signals)
    except ValueError as verr:
        if verr.args == ('Invalid timestamps!',):
            return None
        raise verr

In [17]:
erps = {}

In [18]:
GOOD_SUBJECT_DIRS = []

In [19]:
for s, subject_dir in enumerate(sorted(NWB_SUBJECTS)):
    subject = subject_dir.split('/')[-2]
    if all(os.path.exists(subject_dir + "/" + cond) for cond in CONDITIONS):
        logging.info("Finished epoching of %s" % subject_dir)
        continue
    recording = subjectdir_recording(subject_dir)
    if recording is None:
        continue

    GOOD_SUBJECT_DIRS.append(subject_dir)
    for cond in CONDITIONS:
        condition_epochs = recording.intervals["type"] == cond
        trial_epochs = recording.intervals["type"] == "trial"
        sampling = recording.epoch(condition_epochs, trial_epochs, PRETRIAL_SECONDS, POSTTRIAL_SECONDS)

        os.makedirs(subject_dir + '/' + cond, exist_ok=True)
        sampling.pickle(subject_dir + '/' + cond)
        del sampling
        logging.info("Finished epoching of %s condition in %s" % (cond, subject_dir))
    del recording
    logging.info("Finished epoching of %s" % subject_dir)

/home/eli/miniforge3/envs/ephys/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.6.0 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/home/eli/miniforge3/envs/ephys/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  return func(args[0], **pargs)
/home/eli/miniforge3/envs/ephys/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.3.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)
/home/eli/miniforge3/envs/ephys/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.6.0 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/home/eli/miniforge3/envs/ephys/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namesp

INFO:root:Finished epoching of go_gloexp condition in /mnt/data/DRAFT/000253/sub-632487/


INFO:root:Finished epoching of lo_gloexp condition in /mnt/data/DRAFT/000253/sub-632487/


INFO:root:Finished epoching of go_seqctl condition in /mnt/data/DRAFT/000253/sub-632487/


INFO:root:Finished epoching of lo_rndctl condition in /mnt/data/DRAFT/000253/sub-632487/


...

INFO:root:Finished epoching of go_seqctl condition in /mnt/data/DRAFT/000253/sub-649323/


INFO:root:Finished epoching of lo_rndctl condition in /mnt/data/DRAFT/000253/sub-649323/


INFO:root:Finished epoching of igo_seqctl condition in /mnt/data/DRAFT/000253/sub-649323/


INFO:root:Finished epoching of /mnt/data/DRAFT/000253/sub-649323/


/home/eli/miniforge3/envs/ephys/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.6.0 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/home/eli/miniforge3/envs/ephys/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  return func(args[0], **pargs)
/home/eli/miniforge3/envs/ephys/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.3.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)
